<a href="https://colab.research.google.com/github/michelle-lo/APCS-repo/blob/main/B99_Analysis/Sitcom_Success_Factors_b99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install stop-words

  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=e74632f1af18c15ea3c859622eb28ed842fe89702939a309a0c86d047acbc34e
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop-words


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from textblob import TextBlob
import os.path
import gensim
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

import seaborn as sb
import numpy as np
from textblob import TextBlob

# For matrix operations
import numpy as np
# For data processing
import pandas as pd
# For text processing
import nltk
# For regular expressions
import re
# For dividing numbers
from __future__ import division

# stop words
from stop_words import get_stop_words

# Set seed so we get same random allocation on each run of code
np.random.seed(2017)

In [6]:
url = "https://raw.githubusercontent.com/crystal-zhu/Sitcom_Success_Factors-CSDS_312/main/B99_Analysis/data/B99_Script/Brooklyn99_Season1-4_Dataset.csv"
response = requests.get(url)


# Check if the request was successful (status code 200)
if response.status_code == 200:
    with open("Brooklyn99_Season1-4_Dataset", "w") as f:
        f.write(response.text)
    print("CSV file downloaded successfully")
else:
    print("Failed to download CSV file. Status code:", response.status_code)

df = pd.read_csv("Brooklyn99_Season1-4_Dataset")[0:3000]

CSV file downloaded successfully


In [7]:
df

,name,line
0,JAKE,This job is eating me alive. I can't breathe ...
1,AMY,"Hey! What are you doing, weirdo?"
2,JAKE,I'm doing the best speech from Donnie Brasco....
3,AMY,"Get it together, man. Okay?"
4,AMY,So the store was hit about two hours ago. The...
...,...,...
2995,VULTURE,"Look, Jake, I know that you and Santiago are ..."
2996,JAKE,What? You can't do that. It's completely agai...
2997,VULTURE,"Okay, Sonia Sotomayor. I would never break th..."
2998,JAKE,"No, don't do that, all right? I can learn to ..."


In [8]:
# Load tokenizer function
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

# Tokenizing first observation
tokenize_obs = tokenizer.tokenize(df["line"][0])

# Example of tokenizing first observation
print('Tokenize first observation: \n%s' % tokenize_obs)

# Load list of common stop words

# Create English stop words list
eng_stop = [str(word) for word in get_stop_words('english')]

# Print a few stop words
print('Stop words in english: \n%s' % eng_stop[1:10])


Tokenize first observation: 
['This', 'job', 'is', 'eating', 'me', 'alive', 'I', 'can', 't', 'breathe', 'anymore', 'I', 'spent', 'all', 'these', 'years', 'trying', 'to', 'be', 'the', 'good', 'guy', 'the', 'man', 'in', 'the', 'white', 'hat', 'I', 'm', 'not', 'becoming', 'like', 'them', 'I', 'am', 'them']
Stop words in english: 
['about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and']


In [9]:
# Import function for stemming text
from nltk.stem.porter import PorterStemmer

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

# Text data to itterate over
text_data = [line for line in df["line"] if line != '']

# Convert text data into a list of comments after stop words and stemming are accounted for
for line in range(len(df)):

    # Convert comment all to lower case
    raw_lower = text_data[line].lower()

    # Tokenize comment
    line_token = tokenizer.tokenize(raw_lower)

    # Only keep letters in comments
    clean_token = [re.sub(r'[^a-zA-Z]','', word) for word in line_token]

    # Take out stop words
    stop_token = [word for word in clean_token if not word in eng_stop if word != '']

    # Take out stem words
    stem_token = [str(p_stemmer.stem(word)) for word in stop_token]

    # Replace comment with cleaned list of words
    text_data[line] = stop_token


print(text_data)

[['job', 'eating', 'alive', 'can', 't', 'breathe', 'anymore', 'spent', 'years', 'trying', 'good', 'guy', 'man', 'white', 'hat', 'm', 'becoming', 'like'], ['hey', 'weirdo'], ['m', 'best', 'speech', 'donnie', 'brasco', 'actually', 'ten', 'best', 'speech', 'donnie', 'brasco', 'stares', 'screens', 'sup'], ['get', 'together', 'man', 'okay'], ['store', 'hit', 'two', 'hours', 'ago', 'took', 'mostly', 'tablets', 'laptops', 'cameras'], ['sorry'], ['d', 'like', 'list', 'employees', 'whoever', 'access', 'store', 'd', 'also', 'like', 'apologize', 'partner', 'parents', 'didn', 't', 'give', 'enough', 'attention'], ['uh', 'detective', 'already', 'solved', 'case', 're', 'looking', 'three', 'white', 'males', 'one', 'sleeve', 'tats', 'arms'], ['know'], ['informant', 'inside', 's', 'years', 'watching', 'learning', 'waiting', 'code', 'name', 'holds', 'stuffed', 'bear', 'fuzzy', 'cuddle', 'bear', 's', 'nanny', 'cam', 'flips', 'bear', 'show', 'electronic', 'device', 'inside', 'appears', 'end', 'got', 'lucky

In [10]:
# Need to convert our list of list data into single list
#  if words != ''
words_list = [words for sublist in text_data for words in sublist]

# Vocabulary is the set of unique words used
vocab_total = set(words_list)

# Take a look at few words
print('Few words from vocabulary list: \n%s' % list(vocab_total)[1:10])

# Size of vocabulary list
print('Number of unique words in data: \n%s' % len(vocab_total))


Few words from vocabulary list: 
['gah', 'marijuana', 'regis', 'current', 'wig', 'trash', 'pad', 'scottish', 'attached']
Number of unique words in data: 
4245


In [11]:
# Convert each comment into a vector by replacing the words by their unique ID
text_ID = []

# Loop over cleaned text data
for line in range(len(text_data)):

    # Append comment replaced by unique word IDs
    comment_vector = [list(vocab_total).index(words) for words in text_data[line]]
    text_ID.append(comment_vector)

# Let's check the first comment
print ("The first comment (after processing) is: \n%s" % text_data[0])
print('First comment as a vector of word IDs is: \n%s' % text_ID[0])
# each word is associated with an ID, identify the ID numbers for each word on the text

The first comment (after processing) is: 
['job', 'eating', 'alive', 'can', 't', 'breathe', 'anymore', 'spent', 'years', 'trying', 'good', 'guy', 'man', 'white', 'hat', 'm', 'becoming', 'like']
First comment as a vector of word IDs is: 
[1709, 2644, 1106, 3230, 251, 3289, 1465, 1552, 3806, 2573, 4015, 136, 278, 4212, 1328, 538, 469, 3378]


# LDA Algorithm

In [12]:
# Initialize hyperparameters in LDA

# Dirichlet parameters
# Alpha is the parameter for the prior topic distribution within documents
alpha = 0.2

# Beta is the parameter for the prior topic distribution within documents
beta = 0.001

# Text corpus itterations
corpus_itter = 200

# Number of topics
K = 2

# Vocabulary size
V = len(vocab_total)

# Number of Documents
D = len(text_ID)

# For practical implementation, we will generate the following three count matrices:
# 1) Word-Topic count matrix, 2) Topic-Document assignment matrix, 3) Document-Topic count matrix

# Initialize word-topic count matrix (size K x V, K = # topics, V = # vocabulary)
word_topic_count = np.zeros((K,V))

# Initialize topic-document assignment matrix
topic_doc_assign = [np.zeros(len(sublist)) for sublist in text_ID] #for each "doc", aka line

# Initialize document-topic matrix
doc_topic_count = np.zeros((D,K))

#print(topic_doc_assign)
print(doc_topic_count)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [13]:
# Generate word-topic count matrix with randomly assigned topics

# Loop over documents
for doc in range(D):

    # Loop over words in given document
    for word in range(len(text_ID[doc])):

        # Step 1: Randomly assign topics to each word in document
        # Note random.choice generates number {0,...,K-1}
        topic_doc_assign[doc][word] = np.random.choice(K,1)

        # Record word-topic and word-ID
        word_topic = int(topic_doc_assign[doc][word])
        word_doc_ID = text_ID[doc][word]

        # Increment word-topic count matrix
        word_topic_count[word_topic][word_doc_ID] += 1

# Print word-topic matrix
print('Word-topic count matrix with random topic assignment: \n%s' % word_topic_count)

<ipython-input-13-bd3ec878e4d8>:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  topic_doc_assign[doc][word] = np.random.choice(K,1)


Word-topic count matrix with random topic assignment: 
[[0. 2. 0. ... 1. 1. 0.]
 [1. 3. 1. ... 0. 0. 3.]]


In [14]:
# Generate document-topic count matrix with randomly assigned topics

# Loop over documents (D = numb. docs)
for doc in range(D):

    # Loop over topics (K = numb. topics)
    for topic in range(K):

        # topic-document vector
        topic_doc_vector = topic_doc_assign[doc]

        # Update document-topic count
        doc_topic_count[doc][topic] = sum(topic_doc_vector == topic)

# Print document-topic matrix
print('Subset of document-topic count matrix with random topic assignment: \n%s' % doc_topic_count[0:5])

Subset of document-topic count matrix with random topic assignment: 
[[10.  8.]
 [ 1.  1.]
 [ 6.  8.]
 [ 3.  1.]
 [ 7.  3.]]


In [15]:
# Main part of LDA algorithm (takes a few minutes to run)
# Run through text corpus multiple times
for itter in range(corpus_itter):

    # Loop over all documents
    for doc in range(D):

        # Loop over words in given document
        for word in range(len(text_ID[doc])):

            # Initial topic-word assignment
            init_topic_assign = int(topic_doc_assign[doc][word])

            # Initial word ID of word
            word_id = text_ID[doc][word]

            # Before finiding posterior probabilities, remove current word from count matrixes
            doc_topic_count[doc][init_topic_assign] -= 1
            word_topic_count[init_topic_assign][word_id] -=1

            # Find probability used for reassigning topics to words within documents

            # Denominator in first term (Numb. of words in doc + numb. topics * alpha)
            denom1 = sum(doc_topic_count[doc]) + K*alpha

            # Denominator in second term (Numb. of words in topic + numb. words in vocab * beta)
            denom2 = np.sum(word_topic_count, axis = 1) + V*beta

            # Numerators, number of words assigned to a topic + prior dirichlet param
            numerator1 = [doc_topic_count[doc][col] for col in range(K)]
            numerator1 = np.array(numerator1) + alpha
            numerator2 = [word_topic_count[row][word_id] for row in range(K)]
            numerator2 = np.array(numerator2) + beta

            # Compute conditional probability of assigning each topic
            # Recall that this is obtained from gibbs sampling
            prob_topics = (numerator1/denom1)*(numerator2/denom2)
            prob_topics = prob_topics/sum(prob_topics)

            # Update topic assignment (topic can be drawn with prob. found above)
            update_topic_assign = np.random.choice(K,1,list(prob_topics))
            topic_doc_assign[doc][word] = update_topic_assign

            # Add in current word back into count matrixes
            doc_topic_count[doc][init_topic_assign] += 1
            word_topic_count[init_topic_assign][word_id] +=1

<ipython-input-15-477a05868661>:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  topic_doc_assign[doc][word] = update_topic_assign


In [16]:
# Compute posterior mean of document-topic distribution
theta = (doc_topic_count+alpha)
theta_row_sum = np.sum(theta, axis = 1)
theta = theta/theta_row_sum.reshape((D,1))

# Print document-topic mixture
print('Subset of document-topic mixture matrix: \n%s' % theta[0:10])


Subset of document-topic mixture matrix: 
[[0.55434783 0.44565217]
 [0.5        0.5       ]
 [0.43055556 0.56944444]
 [0.72727273 0.27272727]
 [0.69230769 0.30769231]
 [0.85714286 0.14285714]
 [0.44565217 0.55434783]
 [0.5        0.5       ]
 [0.85714286 0.14285714]
 [0.5        0.5       ]]


In [17]:
# Compute posterior mean of word-topic distribution within documents
phi = (word_topic_count + beta)
phi_row_sum = np.sum(phi, axis = 1)
phi = phi/phi_row_sum.reshape((K,1))


# Print topic-word mixture
print('Topic-word mixture matrix: \n%s' % phi)

Topic-word mixture matrix: 
[[9.14892576e-08 1.83070004e-04 9.14892576e-08 ... 9.15807468e-05
  9.15807468e-05 9.14892576e-08]
 [9.21795208e-05 2.76354387e-04 9.21795208e-05 ... 9.20874333e-08
  9.20874333e-08 2.76354387e-04]]


In [18]:
# Explore the top words that make up each topic

# Initialize list of dictionaries
list_dict_topics = []

# Loop over topics
for topic in range(K):

    # Initialize (vocab,prob) dictionary
    mydict = {}

    # Loop over vocabular
    for word in range(V):

        # Create dictionary {(vocab,prob)}
        mydict[list(vocab_total)[word]] = phi[topic][word]

    # Create list of dictionaries
    list_dict_topics.append(mydict)

In [21]:
print(list_dict_topics)

[{'men': 9.148925756010044e-08, 'gah': 0.00018307000437776098, 'marijuana': 9.148925756010044e-08, 'regis': 0.00018307000437776098, 'current': 9.158074681766053e-05, 'wig': 9.158074681766053e-05, 'trash': 0.0003660485194979619, 'pad': 9.158074681766053e-05, 'scottish': 9.158074681766053e-05, 'attached': 9.158074681766053e-05, 'linen': 9.158074681766053e-05, 'containers': 0.00018307000437776098, 'stenographer': 9.158074681766053e-05, 'skinny': 9.158074681766053e-05, 'worried': 0.00018307000437776098, 'slurping': 9.148925756010044e-08, 'phrase': 9.158074681766053e-05, 'pinot': 9.158074681766053e-05, 'owner': 9.158074681766053e-05, 'robe': 9.158074681766053e-05, 'cassettes': 9.158074681766053e-05, 'reason': 0.00018307000437776098, 'bong': 9.158074681766053e-05, 'bossing': 9.148925756010044e-08, 'coast': 9.158074681766053e-05, 'peace': 0.00027455926193786144, 'boobies': 9.148925756010044e-08, 'guts': 9.158074681766053e-05, 'incompetent': 9.158074681766053e-05, 'enough': 0.00064051629217826

In [19]:
# First topic
# The first 10 words are ignored, because they most overlap with topic 2
# Commonly appearing words in topic 1
sorted([(value,key) for (key,value) in list_dict_topics[0].items()])[::-1][10:30]

[(0.006953275063825194, 'right'),
 (0.006495828776024692, 'okay'),
 (0.006495828776024692, 'get'),
 (0.006404339518464592, 'one'),
 (0.005855403973103989, 'yeah'),
 (0.005855403973103989, 'got'),
 (0.005672425457983787, 'go'),
 (0.005306468427743386, 'well'),
 (0.005306468427743386, 'good'),
 (0.005123489912623185, 'gonna'),
 (0.004757532882382783, 'peralta'),
 (0.004666043624822682, 'jake'),
 (0.004483065109702482, 'hey'),
 (0.004483065109702482, 'captain'),
 (0.004300086594582281, 'think'),
 (0.0037511510492216783, 'yes'),
 (0.0036596617916615777, 'now'),
 (0.0036596617916615777, 'make'),
 (0.0036596617916615777, 'back'),
 (0.003568172534101477, 'll')]

In [20]:
# Second topic
# The first 10 words are ignored, because they most overlap with topic 1
# Commonly appearing words in topic 2
sorted([(value,key) for (key,value) in list_dict_topics[1].items()])[::-1]

[(0.04199196168794424, 's'),
 (0.016760004954303916, 'm'),
 (0.016483742654300554, 't'),
 (0.011418933820905598, 're'),
 (0.011050584087567783, 'just'),
 (0.009393010287547616, 'know'),
 (0.009024660554209802, 'can'),
 (0.00828796108753417, 'like'),
 (0.007827523920861904, 'right'),
 (0.007735436487527449, 'oh'),
 (0.007551261620858542, 'get'),
 (0.006446212420845096, 'okay'),
 (0.006077862687507282, 'one'),
 (0.005525338087500558, 'got'),
 (0.005525338087500558, 'don'),
 (0.005341163220831651, 'let'),
 (0.005249075787497197, 've'),
 (0.005249075787497197, 'go'),
 (0.005156988354162744, 'well'),
 (0.005156988354162744, 'gonna'),
 (0.00506490092082829, 'think'),
 (0.004880726054159382, 'll'),
 (0.004788638620824928, 'good'),
 (0.004696551187490475, 'yeah'),
 (0.004604463754156021, 'hey'),
 (0.004604463754156021, 'captain'),
 (0.0042361140208182066, 'now'),
 (0.0042361140208182066, 'look'),
 (0.004144026587483753, 'want'),
 (0.004051939154149298, 'back'),
 (0.003959851720814845, 'd'),
 (